In [5]:
import os
import json

from p2 import StackingProblem
from collections import defaultdict
from datetime import datetime
import time
import roar_net_api.algorithms as alg

In [6]:

def extract_block_info(data):
    block_info = []

    # Production stack
    if "Production" in data and "BottomToTop" in data["Production"]:
        for i, block in enumerate(data["Production"]["BottomToTop"], start=1):
            block_info.append((
                block["Id"],                       # blockId
                block["Due"]["MilliSeconds"],       # due
                0,                       # stackId
                i                                   # heightOfBlock (bottom=1)
            ))

    # Buffers
    if "Buffers" in data:
        for buffer in data["Buffers"]:
            stack_id = buffer["Id"]
            for i, block in enumerate(buffer.get("BottomToTop", []), start=1):
                block_info.append((
                    block["Id"],                   # blockId
                    block["Due"]["MilliSeconds"] /1000,  # due
                    stack_id,                      # stackId (buffer ID)
                    i                              # heightOfBlock
                ))

    return block_info

def to_jagged(data):
    #(blockid, stackId, height)

    noStacks = max([t[1] for t in data]) + 1
    maxHeight = max([t[2] for t in data])
    jagged = [[None for _ in range(maxHeight)] for _ in range(noStacks)]
    for (blockid, stackId, height) in data:
        jagged[stackId][height-1] = blockid  

    for stack in jagged:
        while stack and stack[-1] is None:
            stack.pop() 
    
    return jagged

def total_seconds(t):
    return t.hour * 3600 + t.minute * 60 + t.second

def time_str_to_speed(time_str: str) -> float:
    # Parse hh:mm:ss into a timedelta
    t = datetime.strptime(time_str, "%H:%M:%S")
    total_seconds = t.hour * 3600 + t.minute * 60 + t.second
    
    if total_seconds == 0:
        raise ValueError("Time per unit cannot be zero.")
    
    return 1.0 / total_seconds

def stacking_problem_from_json(merged_json):
    base = merged_json["base"]
    world = merged_json["world"]

    # Example mapping: adjust field names as needed!
    h1 =  base.get("BufferMaxHeight")
    h2 = base.get("ProductionMaxHeight")
    buffers = base.get("BufferCount")

    max_height = [h2]+[h1]*buffers+[1]
    blocks = list(sorted(extract_block_info(world), key=lambda x: x[0]))
    print(blocks)

    due_dates = [block[1] for block in blocks]
    initial_stacks = to_jagged([(i, t[2],t[3]) for i, t in enumerate(blocks)])
    initial_stacks.append([])  # Add empty handover stack
    print(initial_stacks)
    handover_time = total_seconds(datetime.strptime(base.get("HandoverTimeMean"), "%H:%M:%S"))
    horizontal_speed = time_str_to_speed(base.get("CraneMoveTimeMean"))
    vertical_speed = time_str_to_speed(base.get("HoistMoveTimeMean"))
    crane_height = max(max_height) + 1
    handover_stack = len(initial_stacks) - 1  # last stack is handover


    # You may need to convert types, e.g. lists of floats/ints
    return StackingProblem(
        max_height=max_height,
        due_dates=due_dates,
        handover_time=handover_time,
        horizontal_speed=horizontal_speed,
        vertical_speed=vertical_speed,
        crane_height=crane_height,
        handover_stack=handover_stack,
        initial_stacks=initial_stacks
    )

data_dir = "data"
files = [f for f in os.listdir(data_dir) if f.endswith(".json")]

pairs = []
for f in files:
    if f.endswith("_World.json"):
        base = f.replace("_World.json", ".json")
        if base in files:
            pairs.append((base, f))

for base_file, world_file in pairs:
    base_path = os.path.join(data_dir, base_file)
    world_path = os.path.join(data_dir, world_file)
    with open(base_path, "r", encoding="utf-8") as f:
        base_data = json.load(f)
    with open(world_path, "r", encoding="utf-8") as f:
        world_data = json.load(f)
    merged = {
        "base": base_data,
        "world": world_data
    }
    problem = stacking_problem_from_json(merged)
    with open(world_path.replace("_World","_instance"), "w") as f:
        f.write(problem.to_json())

[(1, 1575.686, 3, 3), (2, 1315.151, 2, 7), (3, 1535.035, 6, 8), (4, 2064.676, 4, 8), (5, 1470.069, 1, 3), (6, 1204.386, 5, 5), (7, 1934.042, 6, 7), (8, 1048.604, 1, 2), (9, 1427.422, 2, 6), (10, 1056.172, 6, 6), (11, 918.999, 4, 7), (12, 762.065, 1, 1), (13, 976.745, 6, 5), (14, 1006.513, 2, 5), (15, 867.015, 5, 4), (16, 1082.974, 3, 2), (17, 1019.891, 2, 4), (18, 670.219, 2, 3), (19, 924.665, 4, 6), (20, 815.906, 5, 3), (21, 995.417, 4, 5), (22, 780.758, 6, 4), (23, 1069.423, 3, 1), (24, 539.129, 5, 2), (25, 201.985, 4, 4), (26, 814.551, 4, 3), (27, 427.279, 6, 3), (28, 404.81, 2, 2), (29, 1366.622, 4, 2), (30, 523.248, 2, 1), (31, 1057.936, 6, 2), (32, 944.107, 6, 1), (33, 134.88, 4, 1), (34, 345.313, 5, 1), (35, 1466904, 0, 1)]
[[34], [11, 7, 4], [29, 27, 17, 16, 13, 8, 1], [22, 15, 0], [32, 28, 25, 24, 20, 18, 10, 3], [33, 23, 19, 14, 5], [31, 30, 26, 21, 12, 9, 6, 2], []]
[(1, 1831.958, 3, 4), (2, 1568.165, 2, 8), (3, 1790.443, 6, 8), (4, 2307.52, 4, 8), (5, 1724.523, 1, 5), (6, 1

In [ ]:
data_dir = "data"
instance_files = [f for f in os.listdir(data_dir) if f.endswith("instance.json")]

for filename in instance_files:
    filepath = os.path.join(data_dir, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        data_str = f.read()
    problem = StackingProblem.from_json(data_str)

    print(f'Testing instance {filename}')
    
    #Test random solution
    start_time = time.time()
    solution_random = problem.random_solution()
    print({ 
        'name':'random',
        'objective': solution_random.objective_value(),
        'time': time.time() - start_time,
        'feasible': solution_random.is_feasible
    })

    #Test greedy construction
    start_time = time.time()
    solution_greedy = alg.greedy_construction(problem)
    print({
        'name':'greedy',
        'objective': solution_greedy.objective_value(),
        'time': time.time() - start_time,
        'feasible': solution_greedy.is_feasible
    })

    #Test beam search
    start_time = time.time()
    solution_beam = alg.beam_search(problem,bw = 1)
    print({
        'name':'beam',
        'objective': solution_beam.objective_value(),
        'time': time.time() - start_time,
        'feasible': solution_beam.is_feasible
    })

    #Test grasp
    start_time = time.time()
    solution_grasp = alg.grasp(problem, 1)
    print({
        'name':'grasp',
        'objective': solution_grasp.objective_value(),
        'time': time.time() - start_time,
        'feasible': solution_grasp.is_feasible
    })

    #Test first improvement
    start_time = time.time()
    solution_fi = alg.first_improvement(problem, solution_greedy)
    print({
        'name':'first improvement',
        'objective': solution_fi.objective_value(),
        'time': time.time() - start_time,
        'feasible': solution_fi.is_feasible
    })

    #Test best improvement
    start_time = time.time()
    solution_bi = alg.best_improvement(problem, solution_greedy)
    print({
        'name':'best improvement',
        'objective': solution_bi.objective_value(),
        'time': time.time() - start_time,
        'feasible': solution_bi.is_feasible
    })

    #Test simulated annealing
    start_time = time.time()
    solution_sa = alg.sa(problem, solution_greedy,10000,1.0)
    print({
        'name':'simulated annealing',
        'objective': solution_sa.objective_value(),
        'time': time.time() - start_time,
        'feasible': solution_bi.is_feasible
    })

Testing instance HS-Training-2022-A-Easy_instance.json
{'name': 'random', 'objective': 7979.791465034999, 'time': 0.0003266334533691406, 'feasible': <bound method Solution.is_feasible of Solution(feasible=True, objval=7979.791465034999, lb=7979.791465034999, moves=35,state=StackingState(time=1530.0, handover_ready=1532.0, stacks=[[], [], [], [], [], [], [], []], overdue=7979.791465034999))>}
{'name': 'greedy', 'objective': 14391.960834874997, 'time': 0.10756683349609375, 'feasible': <bound method Solution.is_feasible of Solution(feasible=True, objval=14391.960834874997, lb=14391.960834874997, moves=71,state=StackingState(time=2074.0, handover_ready=2076.0, stacks=[[], [], [], [], [], [], [], []], overdue=14391.960834874997))>}


KeyboardInterrupt: 